In [1]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
%matplotlib inline

In [2]:
start = datetime.datetime(2020,5,1)
end = datetime.datetime(2022,5,9)

In [3]:
nifty = data.DataReader('^NSEI', 'yahoo', start, end)
nifty.drop(columns=['Adj Close','Volume','High','Low'],inplace=True)
nifty['Index']=range(1,1+len(nifty))
nifty.head()

,Open,Close,Index
Date,,,
2020-05-04,9533.500000,9293.500000,1
2020-05-05,9429.400391,9205.599609,2
2020-05-06,9226.799805,9270.900391,3
2020-05-07,9234.049805,9199.049805,4
2020-05-08,9376.950195,9251.500000,5


In [4]:
nifty_copy=data.DataReader('^NSEI', 'yahoo', start, end)
nifty_copy.drop(columns=['Adj Close','Volume','High','Low','Open'],inplace=True)
nifty_copy['Index']=range(2,2+len(nifty))
nifty_copy.rename({'Close':'Previous_Close'},axis=1,inplace=True)
nifty_copy.head()

,Previous_Close,Index
Date,,
2020-05-04,9293.500000,2
2020-05-05,9205.599609,3
2020-05-06,9270.900391,4
2020-05-07,9199.049805,5
2020-05-08,9251.500000,6


In [5]:
nifty_final=pd.merge(nifty,nifty_copy,on='Index',how='inner')
nifty_final.drop(columns=['Index'],inplace=True)
nifty_final.head()

,Open,Close,Previous_Close
0,9429.400391,9205.599609,9293.500000
1,9226.799805,9270.900391,9205.599609
2,9234.049805,9199.049805,9270.900391
3,9376.950195,9251.500000,9199.049805
4,9348.150391,9239.200195,9251.500000


In [6]:
nifty_final['percent_change']=(((nifty_final['Open']/nifty_final['Previous_Close'])-1)*100)

In [7]:
nifty_final.head()

,Open,Close,Previous_Close,percent_change
0,9429.400391,9205.599609,9293.500000,1.462317
1,9226.799805,9270.900391,9205.599609,0.230297
2,9234.049805,9199.049805,9270.900391,-0.397487
3,9376.950195,9251.500000,9199.049805,1.933900
4,9348.150391,9239.200195,9251.500000,1.044700


In [8]:
nifty_final['Open_prevclose']=nifty_final['Open']-nifty_final['Previous_Close']
nifty_final['close_open']=nifty_final['Close']-nifty_final['Open']
def optic_1(x):
    if x>0:
        return 'Positive'
    elif x<0:
        return 'Negative'
    else:
        return 'Neutral'
nifty_final['Inital']=nifty_final['Open_prevclose'].apply(optic_1)
def optic_2(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0
nifty_final['Final']=nifty_final['close_open'].apply(optic_2)
nifty_final.head()

,Open,Close,Previous_Close,percent_change,Open_prevclose,close_open,Inital,Final
0,9429.400391,9205.599609,9293.500000,1.462317,135.900391,-223.800781,Positive,-1
1,9226.799805,9270.900391,9205.599609,0.230297,21.200195,44.100586,Positive,1
2,9234.049805,9199.049805,9270.900391,-0.397487,-36.850586,-35.000000,Negative,-1
3,9376.950195,9251.500000,9199.049805,1.933900,177.900391,-125.450195,Positive,-1
4,9348.150391,9239.200195,9251.500000,1.044700,96.650391,-108.950195,Positive,-1


In [16]:
nifty_final.groupby(by=["Inital","Final"]).count()

Open  Close  Previous_Close  percent_change  Open_prevclose  \
Inital   Final                                                                
Negative -1       69     69              69              69              69   
          1       76     76              76              76              76   
Positive -1      175    175             175             175             175   
          1      181    181             181             181             181   

                close_open  
Inital   Final              
Negative -1             69  
          1             76  
Positive -1            175  
          1            181

In [106]:
nifty_final['Change']=nifty_final['Close']-nifty_final['Open']
nifty_final['today_percentage']=(((nifty_final['Close']-nifty_final['Open'])/(nifty_final['Open']))*100)
nifty_final.drop(columns=['Open','Close','Previous_Close'],inplace=True)
nifty_final.head()

,percent_change,Change,today_percentage
0,1.462317,-223.800781,-2.373436
1,0.230297,44.100586,0.477962
2,-0.397487,-35.000000,-0.379032
3,1.933900,-125.450195,-1.337857
4,1.044700,-108.950195,-1.165473


In [107]:
conver_disc={'Change':int,'percent_change':float,'today_percentage':float}
nifty_final=nifty_final.astype(conver_disc)
nifty_final.head()

,percent_change,Change,today_percentage
0,1.462317,-223,-2.373436
1,0.230297,44,0.477962
2,-0.397487,-35,-0.379032
3,1.933900,-125,-1.337857
4,1.044700,-108,-1.165473


In [108]:
nifty_final['Absolute']=nifty_final['Change'].apply(abs)
nifty_final.head()

,percent_change,Change,today_percentage,Absolute
0,1.462317,-223,-2.373436,223
1,0.230297,44,0.477962,44
2,-0.397487,-35,-0.379032,35
3,1.933900,-125,-1.337857,125
4,1.044700,-108,-1.165473,108


In [109]:
nifty_final['percent_change'].quantile(0.95)

1.0276470370343072

In [ ]:
#BANK NIFTY percentage change bucket
def percent_change(x):
    if x<-1.5:
        return 1
    elif x>=-1.5 and x<-0.9:
        return 2
    elif x>=-0.9 and x<-0.3:
        return 3
    elif x>=-0.3 and x<0:
        return 4
    elif x>=0 and x<0.2:
        return 5
    elif x>=0.2 and x<0.4:
        return 6
    elif x>=0.4 and x<0.5:
        return 7
    elif x>=0.5 and x<0.7:
        return 8
    elif x>=0.7 and x<0.8:
        return 9
    elif x>=0.8 and x<1.1:
        return 10
    elif x>=1.1 and x<1.55:
        return 11
    else:
        return 12
#BANK NIFTY today percentage change bucket
def today_change(x):
    if x<-1.8:
        return 1
    elif x>=-1.8 and x<-1.5:
        return 2
    elif x>=-1.5 and x<-1:
        return 3
    elif x>=-1 and x<-0.7:
        return 4
    elif x>=-0.7 and x<-0.4:
        return 5
    elif x>=-0.4 and x<-0.1:
        return 6
    elif x>=-0.1 and x<0.2:
        return 7
    elif x>=0.2 and x<0.5:
        return 8
    elif x>=0.5 and x<1:
        return 9
    elif x>=1 and x<1.6:
        return 10
    elif x>=1.6 and x<2:
        return 11
    elif x>=2 and x<2.3:
        return 12
    else:
        return 13
nifty_final['Open Bucket']=nifty_final['percent_change'].apply(percent_change)
nifty_final['Closed Bucket']=nifty_final['today_percentage'].apply(today_change)
nifty_final.head()

In [110]:
#NIFTY percentage change bucket
def percent_change(x):
    if x<-1:
        return 1
    elif x>=-1 and x<-0.5:
        return 2
    elif x>=-0.5 and x<-0.15:
        return 3
    elif x>=-0.15 and x<0:
        return 4
    elif x>=0 and x<0.15:
        return 5
    elif x>=0.15 and x<0.25:
        return 6
    elif x>=0.25 and x<0.4:
        return 7
    elif x>=0.4 and x<0.5:
        return 8
    elif x>=0.5 and x<0.6:
        return 9
    elif x>=0.6 and x<0.8:
        return 10
    elif x>=0.8 and x<1:
        return 11
    elif x>=1 and x<1.5:
        return 12
    else:
        return 13
#NIFTY today percentage change bucket
def today_change(x):
    if x<-1.5:
        return 1
    elif x>=-1.5 and x<-1:
        return 2
    elif x>=-1 and x<-0.7:
        return 3
    elif x>=-0.7 and x<-0.4:
        return 4
    elif x>=-0.4 and x<-0.2:
        return 5
    elif x>=-0.2 and x<0:
        return 6
    elif x>=0 and x<0.15:
        return 7
    elif x>=0.15 and x<0.3:
        return 8
    elif x>=0.3 and x<0.5:
        return 9
    elif x>=0.5 and x<0.9:
        return 10
    elif x>=0.9 and x<1.3:
        return 11
    else:
        return 12
nifty_final['Open Bucket']=nifty_final['percent_change'].apply(percent_change)
nifty_final['Closed Bucket']=nifty_final['today_percentage'].apply(today_change)
nifty_final.head()

,percent_change,Change,today_percentage,Absolute,Open Bucket,Closed Bucket
0,1.462317,-223,-2.373436,223,12,1
1,0.230297,44,0.477962,44,6,9
2,-0.397487,-35,-0.379032,35,3,5
3,1.933900,-125,-1.337857,125,13,2
4,1.044700,-108,-1.165473,108,12,2


In [111]:
import plotly.express as px
fig=px.scatter(nifty_final, x="Open Bucket", y="Closed Bucket")
fig.show()

In [112]:
df=nifty_final[nifty_final['Open Bucket'].isin([7])]['Closed Bucket'].value_counts()
df=((df/df.sum())*100)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
df.iplot(kind='bar')

In [98]:
df.sum()

99.99999999999997